In [ ]:
# # !pip install --upgrade gensim
# !pip install transformers
# # !pip install -U sentence-transformers
# # !pip install kornia
# # # # !pip install "torch==1.7.0"
# !pip install flair
# !pip install captum

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
from collections import Counter
import re

import nltk
import numpy as np
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import cohen_kappa_score, accuracy_score, f1_score, roc_auc_score, precision_score, hamming_loss, jaccard_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer

from tabulate import tabulate

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, hamming_loss, confusion_matrix
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler
import transformers
from transformers import BertTokenizer, BertModel, AdamW
from transformers import get_linear_schedule_with_warmup
import time

import flair
from flair.data import Sentence
from flair.embeddings import TransformerDocumentEmbeddings
from flair.embeddings import SentenceTransformerDocumentEmbeddings

import warnings
import traceback

import logging
logger = logging.getLogger('flair')
logger.setLevel(logging.ERROR)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jamunoz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jamunoz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda:1")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(1))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

flair.device = device
torch.backends.cudnn.enabled=False

There are 2 GPU(s) available.
We will use the GPU: GeForce RTX 2080


In [ ]:
categories_number_words = {
        1: "Apoyo Pedagógico en asignaturas",
        3: "Apoyo pedagógico personal",
        4: "Tutoría entre pares",
        7: "Hacer a la familia partícipe del proceso",
        8: "Apoyo psicóloga(o)",
        9: "Apoyo fonoaudióloga(o)",
        10: "Apoyo Educador(a) Diferencial",
        11: "Apoyo Kinesióloga(o)",
        12: "Apoyo Médico General",
        13: "Apoyo Terapeuta Ocupacional",
        14: "Control Neurólogo",
        15: "Apoyo Interdisciplinario",
        16: "Adecuación curricular de acceso",
        17: "Adecuación curricular de objetivos"
    }
categories_words_number = {v: k for k, v in categories_number_words.items()}

diagnoses_codes = {
    "Trastorno específico del lenguaje": 0,
    "Trastorno por déficit atencional": 1,
    "Dificultad específica de aprendizaje": 2,
    "Discapacidad intelectual": 3,
    "Discapacidad visual": 4,
    "Trastorno del espectro autista": 5,
    "Discapacidad auditiva - Hipoacusia": 6,
    "Funcionamiento intelectual limítrofe": 7,
    "Síndrome de Down": 8,
    "Trastorno motor": 9,
    "Multidéficit": 10,
    "Retraso global del desarrollo": 11
}

diagnoses_keys = list(diagnoses_codes.keys())

def transform_diag_to_array(code):
    arr = np.zeros(len(diagnoses_keys), dtype=int)
    for (index, label) in enumerate(diagnoses_keys):
        if diagnoses_codes[label]==code:
            arr[index] = 1
    return arr

In [ ]:
train_dataset = pd.read_csv('/research/jamunoz/datasets/train_ds.csv', keep_default_na=False)
val_dataset = pd.read_csv('/research/jamunoz/datasets/val_ds.csv', keep_default_na=False)
test_dataset = pd.read_csv('/research/jamunoz/datasets/test_ds.csv', keep_default_na=False)
# train_dataset = pd.read_csv('gdrive/My Drive/magister/train_ds.csv', keep_default_na=False)
# val_dataset = pd.read_csv('gdrive/My Drive/magister/val_ds.csv', keep_default_na=False)
# test_dataset = pd.read_csv('gdrive/My Drive/magister/test_ds.csv', keep_default_na=False)


# Add OHE diagnosis
train_OHE_diags = []
for diag in train_dataset['Encoded Diagnosis']:
    train_OHE_diags.append(transform_diag_to_array(diag))
temp_train_diags_df = pd.DataFrame(train_OHE_diags, columns=diagnoses_keys)
train_dataset = pd.concat([train_dataset, temp_train_diags_df], axis=1)

val_OHE_diags = []
for diag in val_dataset['Encoded Diagnosis']:
    val_OHE_diags.append(transform_diag_to_array(diag))
temp_val_diags_df = pd.DataFrame(val_OHE_diags, columns=diagnoses_keys)
val_dataset = pd.concat([val_dataset, temp_val_diags_df], axis=1)

test_OHE_diags = []
for diag in test_dataset['Encoded Diagnosis']:
    test_OHE_diags.append(transform_diag_to_array(diag))
temp_test_diags_df = pd.DataFrame(test_OHE_diags, columns=diagnoses_keys)
test_dataset = pd.concat([test_dataset, temp_test_diags_df], axis=1)

In [ ]:
# y_keys = list(strat_present.keys())
Y_KEYS = list(categories_words_number.keys())

# df = pd.DataFrame(data=new_dataset_to_export)
# X = df
# Y = df[y_keys]
X_train = train_dataset.drop(Y_KEYS, axis=1)
Y_train = train_dataset[Y_KEYS]
X_val = val_dataset.drop(Y_KEYS, axis=1)
Y_val = val_dataset[Y_KEYS]
X_test = test_dataset.drop(Y_KEYS, axis=1)
Y_test = test_dataset[Y_KEYS]

strats_amounts = {
              'Adecuación curricular de acceso': 2264,
              'Hacer a la familia partícipe del proceso': 2048,
              'Apoyo Interdisciplinario': 1441, 
              'Apoyo Educador(a) Diferencial': 1311,
              'Apoyo pedagógico personal': 1240,
              'Apoyo fonoaudióloga(o)': 378,
              'Apoyo psicóloga(o)': 588,
              'Apoyo Terapeuta Ocupacional': 153,
              'Tutoría entre pares': 350,
              'Control Neurólogo': 63,
              'Apoyo Médico General': 64,
              'Apoyo Kinesióloga(o)': 32,
              'Adecuación curricular de objetivos': 281,
              'Apoyo Pedagógico en asignaturas': 1314
}
most_unbalanced_strategies = [strategy for strategy in Y_KEYS if (
    strats_amounts[strategy] < (len(X_train) + len(X_val) + len(X_test))*0.15 or strats_amounts[strategy] > (len(X_train) + len(X_val) + len(X_test))*0.85)]
less_unbalanced_strategies = [strategy for strategy in Y_KEYS if strategy not in most_unbalanced_strategies]
only_one_strat = [Y_KEYS[0]]

### Dataset

In [ ]:
class AllJoinedObservationsDataset(Dataset):

    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
      data_row = self.data.iloc[idx]
      labels =  data_row[Y_KEYS]

      tensor_labels = torch.tensor(labels, dtype=torch.int)
      tensor_diags = torch.tensor(data_row[diagnoses_keys], dtype=torch.int)

      dict_to_return = dict(
          all_perceptions=data_row['All perceptions'],
          sne_perceptions=data_row['Special Education Teacher Perceptions'],
          st_perceptions=data_row['Speech Therapist Perceptions'],
          p_perceptions=data_row['Psychologist Perceptions'],
          m_perceptions=data_row['Medical Perceptions'],
          labels=tensor_labels,
          diagnostics=tensor_diags
      )

      return dict_to_return

In [ ]:
class AllJoinedObservationsDataset2(Dataset):

    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
      data_row = self.data.iloc[idx]
      labels =  data_row[Y_KEYS]

      tensor_labels = torch.tensor(labels, dtype=torch.int)
      tensor_diags = torch.tensor(data_row[diagnoses_keys], dtype=torch.int)

      dict_to_return = dict(
          all_perceptions=data_row['All perceptions'],
          sne_perceptions=data_row['Special Education Teacher Perceptions'],
          st_perceptions=data_row['Speech Therapist Perceptions'],
          p_perceptions=data_row['Psychologist Perceptions'],
          m_perceptions=data_row['Medical Perceptions'],
          labels=tensor_labels,
          diagnostics=tensor_diags
      )

      return dict_to_return

### Dataloaders

In [ ]:
TRAIN_BATCH_SIZE=1

def my_collate1(batches):
  modified_batches = []
  for batch in batches:
    batch_dict = {}
    for key, value in batch.items():
      batch_dict[key] = value
    modified_batches.append(batch_dict)
  return modified_batches

transformed_train_dataset=AllJoinedObservationsDataset2(
    train_dataset)

transformed_val_dataset=AllJoinedObservationsDataset2(
    val_dataset)

transformed_test_dataset=AllJoinedObservationsDataset2(
    test_dataset)

train_data_loader=DataLoader(
    transformed_train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate1)

val_data_loader=DataLoader(
    transformed_val_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate1)

test_data_loader=DataLoader(
    transformed_test_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate1)

### Utils

In [ ]:
import statistics

def get_results(targets, outputs):
  TN = 0
  TP = 0
  FP = 0
  FN = 0
  for (i, output) in enumerate(outputs):
    if output==0:
      if targets[i]==0:
        TN += 1
      else:
        FN += 1
    else:
      if targets[i]==1:
        TP += 1
      else:
        FP += 1
  return TP, TN, FP, FN

def findMinDiff(arr):
    n = len(arr)
    arr = sorted(arr)
    diff = 0.5
    for i in range(n-1):
        if arr[i+1] - arr[i] > 0 and arr[i+1] - arr[i] < diff:
            diff = arr[i+1] - arr[i]
    return diff

def get_thresholds(targets, outputs):
  best_thresholds = []
  for i in range(len(outputs[0])):
    real_preds = outputs[:, i]
    trues = targets[:, i]
    max_g = 0
#     max_f1 = 0
    delta_threshold = 0.0001 # findMinDiff(real_preds)*0.9
    positive_ratio = sum(trues)/len(trues)
#     print('pr: ', positive_ratio)
    if positive_ratio > 0.6:
      local_best = 0
      curr_threshold = min(real_preds)
#       print(curr_threshold)
      while curr_threshold < 1:
        preds = [1 if pred > curr_threshold else 0 for pred in real_preds]
        tp, tn, fp, fn = get_results(trues, preds)
        recall = tp/(tp+fn)
        specificity = tn/(tn+fp)
        g_mean = np.sqrt(recall*specificity)
#         f1 = f1_score(trues, preds)
#         print(f1, max_f1, curr_threshold, local_best, tp, tn, fp, fn)
        if tp < tn:
          break
        if g_mean > max_g:
#         if f1 > max_f1:
          max_g = g_mean
#           max_f1 = f1
          local_best = curr_threshold
        curr_threshold += delta_threshold
      best_thresholds.append(local_best)
    elif positive_ratio < 0.4:
      local_best = 1
      curr_threshold = max(real_preds)
      while curr_threshold > 0:
        preds = [1 if pred > curr_threshold else 0 for pred in real_preds]
        tp, tn, fp, fn = get_results(trues, preds)
        recall = tp/(tp+fn)
        specificity = tn/(tn+fp)
        g_mean = np.sqrt(recall*specificity)
#         f1 = f1_score(trues, preds)
        if tn < tp:
          break
        if g_mean > max_g:
#         if f1 > max_f1:
          max_g = g_mean
#           max_f1 = f1
          local_best = curr_threshold
        curr_threshold -= delta_threshold
      best_thresholds.append(local_best)
    else:
      local_best = 0.5
      best_thresholds.append(local_best)
  return best_thresholds

def get_individual_threshold(target, output):
    real_preds = output
    trues = target
    max_g = 0
    # max_f1 = 0
    delta_threshold = 0.0001 # findMinDiff(real_preds)*0.9
    positive_ratio = sum(trues)/len(trues)
#     print('pr: ', positive_ratio)
    if positive_ratio > 0.5:
      local_best = 0
      curr_threshold = min(real_preds)
#       print(curr_threshold)
      while curr_threshold < 1:
        preds = [1 if pred > curr_threshold else 0 for pred in real_preds]
        tp, tn, fp, fn = get_results(trues, preds)
        recall = tp/(tp+fn)
        specificity = tn/(tn+fp)
        g_mean = np.sqrt(recall*specificity)
        # f1 = f1_score(trues, preds)
#         print(f1, max_f1, curr_threshold, local_best, tp, tn, fp, fn)
        if tp < tn:
          break
        if g_mean > max_g:
        # if f1 > max_f1:
          max_g = g_mean
          # max_f1 = f1
          local_best = curr_threshold
        curr_threshold += delta_threshold
      return local_best
    else:
      local_best = 1
      curr_threshold = max(real_preds)
      while curr_threshold > 0:
        preds = [1 if pred > curr_threshold else 0 for pred in real_preds]
        tp, tn, fp, fn = get_results(trues, preds)
        recall = tp/(tp+fn)
        specificity = tn/(tn+fp)
        g_mean = np.sqrt(recall*specificity)
        # f1 = f1_score(trues, preds)
        if tn < tp:
          break
        if g_mean > max_g:
        # if f1 > max_f1:
          max_g = g_mean
          # max_f1 = f1
          local_best = curr_threshold
        curr_threshold -= delta_threshold
      return local_best
    # else:
    #   local_best = 0.5
    #   return local_best

In [ ]:
def loss_fun(outputs, targets):
    loss = nn.BCEWithLogitsLoss()
    # loss = BinaryFocalLossWithLogits(alpha=0.25, reduction='mean')
    try:
      return loss(outputs, targets)
    except Exception:
      print(outputs, targets)
      traceback.print_exc()
    # return nn.BCEWithLogitsLoss()(outputs, targets)

def individual_evaluation_s(target, predicted):
  individual = {}
  for i in range(len(target[0])):
    temp_t = target[:, i]
    temp_p = predicted[:, i]
    diction = dict(
        accuracy=accuracy_score(temp_t, temp_p),
        f1=f1_score(temp_t, temp_p)
    )
    individual[str(i)] = diction
  return individual

def evaluate(target, predicted):
    thresholds = get_thresholds(target, predicted)
    print('thresholds: ', thresholds)
    true_predicted = np.array([[1 if val > thresholds[i] else 0 for (i, val) in enumerate(pred)] for pred in predicted])
    accuracy = accuracy_score(target, true_predicted)
    macro_f1 = f1_score(target, true_predicted, average='macro')
    micro_f1 = f1_score(target, true_predicted, average='micro')
    weighted_f1 = f1_score(target, true_predicted, average='weighted')
    hl = hamming_loss(target, true_predicted)
    macro_js = jaccard_score(target, true_predicted, average="macro")
    micro_js = jaccard_score(target, true_predicted, average="micro")
    individual = individual_evaluation_s(target, true_predicted)
    return {
        "accuracy": accuracy,
        "jaccard_score_macro": macro_js,
        "jaccard_score_micro": micro_js,
        "macro-f1": macro_f1,
        "micro-f1": micro_f1,
        "Hamming Loss": hl,
        "Individual": individual
    }

def evaluate_2(target, predicted):
    true_predicted = predicted
    accuracy = accuracy_score(target, true_predicted)
    macro_f1 = f1_score(target, true_predicted, average='macro')
    micro_f1 = f1_score(target, true_predicted, average='micro')
    hl = hamming_loss(target, true_predicted)
    macro_js = jaccard_score(target, true_predicted, average="macro")
    micro_js = jaccard_score(target, true_predicted, average="micro")
    individual = individual_evaluation_s(target, true_predicted)
    return {
        "accuracy": accuracy,
        "jaccard_score_macro": macro_js,
        "jaccard_score_micro": micro_js,
        "macro-f1": macro_f1,
        "micro-f1": micro_f1,
        "Hamming Loss": hl,
        "Individual": individual
    }

def individual_evaluation(target, predicted):
    threshold = get_individual_threshold(target, predicted)
    print('threshold: ',threshold)
    true_predicted = np.array([1 if val > threshold else 0 for val in predicted])
    default_true_predicted = np.array([1 if val > 0.5 else 0 for val in predicted])
    accuracy = accuracy_score(target, true_predicted)
    f1 = f1_score(target, true_predicted)
    tp, tn, fp, fn = get_results(target, true_predicted)
    recall = tp/(tp+fn)
    specificity = tn/(tn+fp)
    pr = sum(target)/len(target)

    default_accuracy = accuracy_score(target, default_true_predicted)
    default_f1 = f1_score(target, default_true_predicted)
    tp, tn, fp, fn = get_results(target, default_true_predicted)
    default_recall = tp/(tp+fn)
    default_specificity = tn/(tn+fp)
    return {
        "Positive Rate": pr,
        "threshold": threshold[0],
        "accuracy": accuracy,
        "f1": f1,
        "recall": recall,
        "specificity": specificity,
        "default_accuracy": default_accuracy,
        "default_f1": default_f1,
        "default_recall": default_recall,
        "default_specificity": default_specificity,
    }

In [ ]:
def individual_test(target, predicted, threshold):
    true_predicted = np.array([1 if val > threshold else 0 for val in predicted])
    default_true_predicted = np.array([1 if val > 0.5 else 0 for val in predicted])
    accuracy = accuracy_score(target, true_predicted)
    f1 = f1_score(target, true_predicted)
    tp, tn, fp, fn = get_results(target, true_predicted)
    recall = tp/(tp+fn)
    specificity = tn/(tn+fp)
    pr = sum(target)/len(target)

    default_accuracy = accuracy_score(target, default_true_predicted)
    default_f1 = f1_score(target, default_true_predicted)
    tp, tn, fp, fn = get_results(target, default_true_predicted)
    default_recall = tp/(tp+fn)
    default_specificity = tn/(tn+fp)
    return {
        "Positive Rate": pr,
        "accuracy": accuracy,
        "f1": f1,
        "recall": recall,
        "specificity": specificity,
        "default_accuracy": default_accuracy,
        "default_f1": default_f1,
        "default_recall": default_recall,
        "default_specificity": default_specificity,
        "target": target,
        "predicted": true_predicted
    }

### Models

In [ ]:
class Single_Flair_Model_Divided(nn.Module):
    """ A Model for bert fine tuning """

    def __init__(self, n_diags):
        super(Single_Flair_Model_Divided, self).__init__()
        self.embedding = TransformerDocumentEmbeddings("dccuchile/bert-base-spanish-wwm-cased", fine_tune=True, layers='-1')
        # self.lstm = nn.LSTM(768*4, 100, num_layers=1, bidirectional=True)
        self.out = nn.Linear(768*4 + n_diags, 1)
        # self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(0.3)

    def forward(self, sentence_snt_inp, sentence_st_inp, sentence_p_inp, sentence_m_inp, diagnostics):
        self.embedding.embed(sentence_snt_inp)
        self.embedding.embed(sentence_st_inp)
        self.embedding.embed(sentence_p_inp)
        self.embedding.embed(sentence_m_inp)
        emb = [torch.cat([
                          sentence_snt_inp[i].get_embedding(), 
                          sentence_st_inp[i].get_embedding(),
                          sentence_p_inp[i].get_embedding(),
                          sentence_m_inp[i].get_embedding()]) for i in range(len(sentence_snt_inp))]
        # b = torch.stack([torch.stack(emb)])
        # b = b.transpose(0, 1)
        #packed_output, (h_t, h_c) = self.lstm(b, )
        # h_t = h_t.view(-1, 100)
        # output = torch.cat((h_t[0], diagnostics), dim=0)
        output = torch.stack(emb)
        output = torch.cat((output, diagnostics), dim=1)
        output = self.out(output)
        return output

In [ ]:
def individual_eval_loop_fun1(data_loader, model, device, label_index=0):
    model.eval()
    fin_targets = []
    fin_outputs = []
    losses = []
    for batch_idx, batch in enumerate(data_loader):
        text_snt = [Sentence(data["sne_perceptions"]) for data in batch]
        text_st = [Sentence(data["st_perceptions"]) for data in batch]
        text_p = [Sentence(data["p_perceptions"]) for data in batch]
        text_m = [Sentence(data["m_perceptions"]) for data in batch]
        
        labels = [data["labels"][label_index] for data in batch]
        targets = []
        if len(labels) > 1:
            for label_set in labels:
              miniset = []
              for label in label_set:
                miniset.append(torch.tensor([label]))
              targets.append(torch.stack(miniset))
        else:
            miniset = [torch.tensor([labels[0]])]
            targets.append(torch.stack(miniset))
        diagnostics = [data["diagnostics"] for data in batch]

        # text = torch.cat(text)
        targets = torch.cat(targets)
        diagnostics = torch.cat(diagnostics)
        diagnostics = torch.stack([diagnostics])

        # ids = text.to(device, dtype=torch.long)
        # mask = mask.to(device, dtype=torch.long)
        # token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)
        diagnostics = diagnostics.to(device, dtype=torch.long)

        with torch.no_grad():
            outputs = model(
              sentence_snt_inp=text_snt, 
              sentence_st_inp=text_st, 
              sentence_p_inp=text_p, 
              sentence_m_inp=text_m, diagnostics=diagnostics)
            loss = loss_fun(outputs, targets)
            losses.append(loss.item())
            
        outputs = torch.sigmoid(outputs)
        fin_targets.append(targets.cpu().detach().numpy())
        fin_outputs.append(outputs.cpu().detach().numpy())
    return np.concatenate(fin_outputs), np.concatenate(fin_targets), losses

In [ ]:
def individual_test_loop_fun1(data_loader, model, device, label_index=0):
    model.eval()
    fin_targets = []
    fin_outputs = []
    for batch_idx, batch in enumerate(data_loader):
        text_snt = [Sentence(data["sne_perceptions"]) for data in batch]
        text_st = [Sentence(data["st_perceptions"]) for data in batch]
        text_p = [Sentence(data["p_perceptions"]) for data in batch]
        text_m = [Sentence(data["m_perceptions"]) for data in batch]
        
        labels = [data["labels"][label_index] for data in batch]
        targets = []
        if len(labels) > 1:
            for label_set in labels:
              miniset = []
              for label in label_set:
                miniset.append(torch.tensor([label]))
              targets.append(torch.stack(miniset))
        else:
            miniset = [torch.tensor([labels[0]])]
            targets.append(torch.stack(miniset))
        diagnostics = [data["diagnostics"] for data in batch]

        # text = torch.cat(text)
        targets = torch.cat(targets)
        diagnostics = torch.cat(diagnostics)
        diagnostics = torch.stack([diagnostics])

        # ids = text.to(device, dtype=torch.long)
        # mask = mask.to(device, dtype=torch.long)
        # token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)
        diagnostics = diagnostics.to(device, dtype=torch.long)

        with torch.no_grad():
            outputs = model(
              sentence_snt_inp=text_snt, 
              sentence_st_inp=text_st, 
              sentence_p_inp=text_p, 
              sentence_m_inp=text_m, diagnostics=diagnostics)
            
        outputs = torch.sigmoid(outputs)
        fin_targets.append(targets.cpu().detach().numpy())
        fin_outputs.append(outputs.cpu().detach().numpy())
    return np.concatenate(fin_outputs), np.concatenate(fin_targets)

In [ ]:
labels_to_ignore = [7, 8, 9, 10]
results = []
targets = []
for (index, label) in enumerate(Y_KEYS):
  if index not in labels_to_ignore:
    LABEL_INDEX = index
    model = torch.load("/research/jamunoz/models/flair_fine_tuning/d_model_ft_"+str(LABEL_INDEX)+".pt", map_location=device)
    output, target, val_losses_tmp=individual_eval_loop_fun1(val_data_loader, model, device, label_index=LABEL_INDEX)
    tmp_evaluate=individual_evaluation(target, output)
    threshold = tmp_evaluate["threshold"]

    output, target=individual_test_loop_fun1(test_data_loader, model, device, label_index=LABEL_INDEX)
    tmp_test_t=individual_test(target, output, threshold)
    tmp_test_5=individual_test(target, output, 0.5) # threshold)
    if tmp_test_t["accuracy"] > tmp_test_5["accuracy"]:
        tmp_test = tmp_test_t
    else:
        tmp_test = tmp_test_5
    test_target = tmp_test["target"]
    test_predicted = tmp_test["predicted"]

    results.append(test_predicted)
    targets.append(test_target)
    del model
    torch.cuda.empty_cache()

total_results = np.stack(results, axis=-1)
total_targets = np.stack(targets, axis=-1)

threshold:  [0.33395168]
threshold:  [0.37562674]
threshold:  [0.03216672]
threshold:  [0.83679295]
threshold:  [0.1362245]
threshold:  [0.04646888]
threshold:  [0.41304797]
threshold:  [0.597868]
threshold:  [0.8252189]
threshold:  [0.01122122]


In [ ]:
temp_total_targets = []
for el in total_targets:
    temp_total_targets.append(el[0])
total_targets = np.array(temp_total_targets)

In [ ]:
evaluate_2(total_targets, total_results)

{'accuracy': 0.05405405405405406,
 'jaccard_score_macro': 0.29811319481671095,
 'jaccard_score_micro': 0.48853132488873674,
 'macro-f1': 0.3781469942216553,
 'micro-f1': 0.6563937442502299,
 'Hamming Loss': 0.25236486486486487,
 'Individual': {'0': {'accuracy': 0.706081081081081, 'f1': 0.6588235294117648},
  '1': {'accuracy': 0.5878378378378378, 'f1': 0.17006802721088435},
  '2': {'accuracy': 0.8817567567567568, 'f1': 0.0},
  '3': {'accuracy': 0.7483108108108109, 'f1': 0.8396124865446718},
  '4': {'accuracy': 0.8023648648648649, 'f1': 0.0},
  '5': {'accuracy': 0.8733108108108109, 'f1': 0.0},
  '6': {'accuracy': 0.5658783783783784, 'f1': 0.5834683954619125},
  '7': {'accuracy': 0.6368243243243243, 'f1': 0.660347551342812},
  '8': {'accuracy': 0.768581081081081, 'f1': 0.8691499522445081},
  '9': {'accuracy': 0.9054054054054054, 'f1': 0.0}}}